In [1]:
import os
import pandas as pd
import os; os.chdir('../data/data-0520/')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## sample

```json
{
    "relevant APIs": 
        [{"api_id": "0", "api_name": "查询代码", "required_parameters": [["三羊马"]], "rely_apis": [], "tool_name": "股票查询"},
        {"api_id": "1", "api_name": "查询最高价", "required_parameters": ["api_0的结果", "今日"], "rely_apis": ["0"], "tool_name": "股票查询"},
        {"api_id": "2", "api_name": "除法计算", "required_parameters": ["1000000", "api_1的结果"], "rely_apis": ["1"], "tool_name": "数值计算"}],
    "result": ["api_2的结果"]
}
```

## 特征抽取

In [2]:
def get_features(label):
    features = {}
    features['api_list'] = []
    features['api_num'] = 0
    features['result'] = []
    features['true_product'] = ''
    label_d = eval(label)
    for api in label_d['relevant APIs']:
        features['api_list'].append((api['tool_name'],api['api_name']))
        features["api_num"] += 1
        if api['api_name'] == '查询代码':
            features['true_product'] = api['required_parameters'][0][0]
    features['result'] = ','.join([res[:-3] for res in label_d['result']])
    return pd.Series(features)

In [3]:
train_df = pd.read_excel('train.xlsx')
train_df = pd.concat([train_df,train_df.label.apply(get_features)],axis=1)
# train_df.drop(columns=['label'],inplace=True)

---

## CoT 数据准备

### 1. 标准产品名召回

In [7]:
import difflib
import pandas as pd
from tqdm import tqdm

class ProductsLoader:
    def __init__(self,data_dir = './') -> None:
        self.data_dir = data_dir
        self.standard_name = self.load_standard_name()

    def load_standard_name(self):
        data_stock = pd.read_excel(f'{self.data_dir}/标准名.xlsx',sheet_name='股票标准名')
        data_fund = pd.read_excel(f'{self.data_dir}/标准名.xlsx',sheet_name='基金标准名')
        standard_name = data_stock['标准股票名称'].to_list()+data_fund['标准基金名称'].to_list()
        return standard_name
    
    def addProducts2df(self,df):
        df['recall_product'] = None
        print('正在处理...')
        for index,row in tqdm(df.iterrows(),total=df.shape[0]):
            query = row['query']
            products = difflib.get_close_matches(query,self.standard_name,n=50,cutoff=0.0001)
            df.at[index,'recall_product'] = products
            
    def run(self,df):
        self.addProducts2df(df)
        print('处理完成！')

if os.path.exists('../data4cot/train_with_products.xlsx'):
    train_df['recall_product'] = pd.read_excel('train_complete.xlsx')['recall_product'].apply(lambda x:eval(x))
else:
    p=ProductsLoader()
    p.run(train_df)
    train_df.to_excel('../data4cot/train_with_products.xlsx',index=False)

In [8]:
get_index = lambda row:row['recall_product'].index(row['true_product']) if row['true_product'] in row['recall_product'] else -1
train_df['recall_idx'] = train_df.apply(get_index,axis=1)

In [9]:
valid_df = train_df.query('recall_idx!=-1')
valid_df.recall_idx.describe(percentiles=[0.5,0.75,0.9,0.95,0.99,0.999]).astype(int)

count    2868
mean        3
std         7
min         0
50%         0
75%         2
90%        10
95%        20
99%        41
99.9%      47
max        49
Name: recall_idx, dtype: int64

In [10]:
founded_num = train_df.query('recall_idx!=-1').shape[0]
not_found_num = train_df.query('recall_idx==-1 and true_product!=""').shape[0]
no_need_num = train_df.query('true_product==""').shape[0]
founded_num,not_found_num,no_need_num

(2868, 561, 2031)

可以发现，
- 前十个名字已召回90%，二十个召回率95%
- 仍有 561/3000 在n=50时未能召回

### 2. 拼接prompt

In [30]:
n_product = 10
train_df['query_with_products'] = train_df.apply(lambda row:row['query']+'问题中提到的产品标准名可能是：'+'、'.join(row['recall_product'][:n_product]),axis=1)

### 3. label前处理(配合prompt背景)
1. 去掉api名称中的`查询`二字
2. 查询与条件类对齐
  - 条件选基中：基金份额类型(A、B、C) => 基金份额类型
  - 条件选股中：每股经营性现资金流 => 每股经营性现金流

In [37]:
train_df["label"] = train_df["label"].str.replace("\"api_name\": \"查询","\"api_name\": \"")
train_df["label"] = train_df["label"].str.replace("基金份额类型(A、B、C)","基金份额类型")
train_df["label"] = train_df["label"].str.replace("每股经营性现资金流","每股经营性现金流")

In [38]:
train_df[['query_with_products','label']].to_csv('../data4cot/train_cot.csv',index=False)

### 4. 生成带CoT

In [ ]:
%%time
import os
from openai import OpenAI
from prompt import cot_prompt
import pandas as pd
# import os; api_key = os.getenv('DEEPSEEK_API')
api_key = 'sk-b75d2229e28a49dd9f4c1caf4293345c'
client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

src_file = '../data4cot/train_cot.csv'
dest_file = '../data4cot/train_cot.xlsx'

if os.path.exists(dest_file):
    train_df = pd.read_excel(dest_file,dtype=str)
else:
    train_df = pd.read_csv(src_file)
    train_df['cot'] = ''

In [ ]:
train_df.cot = train_df.cot.fillna('')
for i,row in train_df.iterrows():
    # if i < 1300:
    #     continue
    if row['cot'] != '':
        continue
    if i%100==0:
        print(i)
        train_df.to_excel(dest_file, index=False)
    query = row['query_with_products']
    label = row['label']
    prompt = cot_prompt.replace('<QUERY>', query).replace('<LABEL>', label)
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(model="deepseek-chat", messages=messages, max_tokens=4096)   
    train_df.loc[i,'cot'] = response.choices[0].message.content
    # break
train_df.to_excel(dest_file, index=False)

1300
1400
1500


### 5. 生成训练数据

In [3]:
import pandas as pd
from prompt import cot_prompt_train
train_df = pd.read_excel('../data4cot/train_cot.xlsx')

# 


In [7]:
mask = train_df.cot.str.strip().str[:4]!='思考过程'
train_df.loc[mask,'cot'] = '思考过程：\n    ' + train_df.loc[mask,'cot']

In [11]:
train_df['cot_label'] = train_df.cot + '\n\t\t于是最终标准的json格式结果为:\n\t\t\t' + train_df.label
train_df['cot_prompt'] = train_df.query_with_products.apply(lambda query:cot_prompt_train.replace('<QUERY>',query))
train_df[['cot_prompt','cot_label']].to_excel('../data4cot/cot4train.xlsx',index=False)
# for backward compatibility, you can still use `https://api.deepseek.com/v1` as `base_url`.
